The youtube source is: https://www.youtube.com/watch?v=HRvyei7vFSM <br>
The github source: https://github.com/svpino/llm

In [1]:
print("In The Name of Allah most Merci Most Merciful")

In The Name of Allah most Merci Most Merciful


In [94]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL="gpt-3.5-turbo" #this is a chatmodel and it is ment for messegins
#MODEL="mixtral:8x7b"
#MODEL="llama2" #this is a complete model not a chatbot like gpt-3.5-trubo, that is why we get a string as a response

Now we are going to use LangChain

In [95]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import  OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import  OllamaEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings=OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings=OllamaEmbeddings()

model.invoke("What is LLM?")


AIMessage(content='LLM stands for Master of Laws, which is an advanced law degree typically pursued by individuals who have already completed a Juris Doctor (JD) or equivalent law degree. The LLM program allows students to specialize in a particular area of law or jurisdiction, and is often pursued by legal professionals looking to enhance their expertise and career opportunities.', response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 12, 'total_tokens': 79}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-77eab049-49f3-4008-9cd6-05d0f6509de9-0')

In [97]:
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()

chain= model | parser # take the model and type the ouput of this model into the input of the nexe component which is the parser
chain.invoke("What is LLM?")

'LLM stands for Master of Laws, which is an advanced degree in law typically pursued by individuals who have already completed a Juris Doctor (JD) or equivalent law degree. The LLM program allows students to specialize in a particular area of law and gain advanced knowledge and expertise in that area.'

In [114]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader("advisses.pdf") #split the entire pdf documents into different pages, and load each one of these pages into memory
pages=loader.load_and_split()
pages

[Document(page_content='4/5/24, 1:27 AM Student Transcript\nhttps://web.du.edu.om:4443/pls/apex/r/duapex/sis/six-stu-transcript?p214_app_id_p=23385&p214_adv_lst_flg_p=Y&p214_inst_mnu_p=Y&p214_stu_… 1/4T ranscript\nID: 201910442 Muna Said Mubarak Raboob Alshahri\nFaculty: College of Arts and Applied Sciences Major: Bachelor of Science in Computer Science\nNationality: OmaniPlace of\nBirth:Birth\nDate:31-05-\n2001\nAddress: Salalah P.O.Box: Semester:Fall 19-\n20\nSchool:Al Saadah for Basic\nEducation for GirlsYear: 2019 Certificate:General Education\nDiplomaPerc.: 76.56Earn\nCr:110\nNormal Exams\nCambridge University Online Placement\nTest20\nIT Placement Test 36\nMath Placement Test 46\nAdvisor: Nasser T abook\nTerm1 2019-2020 (FP1)\nCourse\nNoCourse T itle Crd# Grade\nFPE 101AFoundation Program\nEnglish-Level 10 57\nFPM 100Pre-Foundation\nMathematics Program0 94\nAtt. Cr Pass Cr .Points MGP AGPA\nSemester: 0 0 0 0 0\nCumulative: 0 0 0 0 0Advisor: Nasser T abook\nTerm2 2019-2020 (FP2)\n

In [116]:
# Now we are going to create a template, or a prompt template, which means ask the model to answe questions from specific context
from langchain.prompts import PromptTemplate

template="""
Answer the question base on the context below. If you can't answer the questions, reply "I don't know".

Context: {context}

Question:{question}
"""

prompt= PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question base on the context below. If you can't answer the questions, reply "I don't know".

Context: Here is some context

Question:Here is a question



In [99]:
chain = prompt | model | parser # thsi means the output of the prompt will be the input of the model and the output of the model will be input for the parser

In [100]:
chain.invoke(
    {
     "context":"The name I was given was is Nasser ",
     "question":"What is my name"   
    }
)

'Your name is Nasser.'

In [117]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

#This will going to create a vector store in memory
#Here we passing the pages which we generated from the pdf documents


In [118]:
retriever= vectorstore.as_retriever()

retriever.invoke("student list", top_k=2)

[Document(page_content='4/5/24, 1:35 AM Student Transcript\nhttps://web.du.edu.om:4443/pls/apex/r/duapex/sis/six-stu-transcript?p214_app_id_p=31437&p214_adv_lst_flg_p=Y&p214_inst_mnu_p=Y&p214_stu_… 2/2Release 1.0  CustomizeCumulative: 0 0 0 0 0\n\ue012\nAdvisor S tudent List Student T ranscript \\ \\\n\xa0\xa0S tu Home\uf015 English\uf1ab Change Semester [Spring 23-24]\uf044', metadata={'source': 'advisses.pdf', 'page': 43}),
 Document(page_content='4/5/24, 1:27 AM Student Transcript\nhttps://web.du.edu.om:4443/pls/apex/r/duapex/sis/six-stu-transcript?p214_app_id_p=23385&p214_adv_lst_flg_p=Y&p214_inst_mnu_p=Y&p214_stu_… 4/4Release 1.0  Customize\n\ue012\nAdvisor S tudent List Student T ranscript \\ \\\n\xa0\xa0S tu Home\uf015 English\uf1ab Change Semester [Spring 23-24]\uf044', metadata={'source': 'advisses.pdf', 'page': 3}),
 Document(page_content='4/5/24, 1:42 AM Student Transcript\nhttps://web.du.edu.om:4443/pls/apex/r/duapex/sis/six-stu-transcript?p214_app_id_p=37506&p214_adv_lst_f

In [112]:
from operator import itemgetter

#
chain=(
    {
        "context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | prompt
    | model
    | parser
)

chain.invoke({"question":"how many students are their"})

"I don't know."

In [113]:
questions=[
    "How many students are there",
    "list the student with their ID and Names and GPA",
    "list the students will GPA less than 70"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({"question":question})}")

Question: How many students are there
Answer: I don't know.
Question: list the student with their ID and Names and GPA
Answer: I don't know.
Question: list the students will GPA less than 70
Answer: I don't know.
